## 跟踪回调
此模块重新组合跟踪在每个时期结束时计算的度量之一的回调，以便对训练做出一些决定。 为了显示使用示例，我们将使用我们的MNIST样本和一个简单的cnn模型。

In [1]:
from fastai.vision import *
from fastai.basics import *
from fastai.callbacks import *

In [2]:
path = untar_data(URLs.MNIST_SAMPLE)
data = ImageDataBunch.from_folder(path)

### class TerminateOnNaNCallback
`TerminateOnNaNCallback() :: Callback`

如果丢失是NaN，则终止训练的回调。

有时候，培训会出现分歧，损失也会减少。 在这种情况下，没有必要继续，所以这个回调会停止训练。

In [3]:
model = simple_cnn((3,16,16,2))
learn = Learner(data, model, metrics=[accuracy])
learn.fit_one_cycle(1,1e4)

epoch,train_loss,valid_loss,accuracy,time
0,nan,nan,0.495584,01:03


使用它可以防止这种情况发生。

In [4]:
model = simple_cnn((3,16,16,2))
learn = Learner(data, model, metrics=[accuracy], callbacks=[TerminateOnNaNCallback()])
learn.fit(2,1e4)

epoch,train_loss,valid_loss,accuracy,time


Epoch/Batch (0/5): Invalid loss, terminating training.
Epoch/Batch (0/6): Invalid loss, terminating training.


### Callback methods
你不要自己调用 - 他们会被fastai的Callback系统自动调用，以启用该类的功能。

<b>on_batch_end</b>

`on_batch_end(last_loss, epoch, num_batch, **kwargs:Any)`

测试last_loss是否为NaN并中断训练。

<b>on_epoch_end</b>

`on_epoch_end(**kwargs:Any)`

在一个epoch结束时被召唤。

### class EarlyStoppingCallback
`EarlyStoppingCallback(learn:Learner, monitor:str='valid_loss', mode:str='auto', min_delta:int=0, patience:int=0) :: TrackerCallback`

TrackerCallback在监控数量停止改善时终止培训。

此回调在学习培训期间跟踪监视器中的数量。 模式可以强制为“最小”或“最大”，但会自动尝试确定数量应尽可能低（验证损失）还是最高可能（精确度）。 如果数量没有通过min_delta改善，将在耐心时期后停止训练。

In [5]:
model = simple_cnn((3,16,16,2))
learn = Learner(data, model, metrics=[accuracy], 
                callback_fns=[partial(EarlyStoppingCallback, monitor='accuracy', min_delta=0.01, patience=3)])
learn.fit(50,1e-42)

epoch,train_loss,valid_loss,accuracy,time
0,0.698448,0.698129,0.504416,00:59
1,0.697836,0.698129,0.504416,00:59
2,0.699521,0.698129,0.504416,01:00
3,0.698427,0.698129,0.504416,01:00


Epoch 4: early stopping


### Callback methods

你不要自己调用 - 他们会被fastai的Callback系统自动调用，以启用该类的功能。

<b>on_train_begin</b>

`on_train_begin(**kwargs:Any)`

初始化内部参数。

<b>on_epoch_end</b>

`on_epoch_end(epoch, **kwargs:Any)`

将监控的值与其最佳分数进行比较，并可能停止培训。

### class SaveModelCallback

`SaveModelCallback(learn:Learner, monitor:str='valid_loss', mode:str='auto', every:str='improvement', name:str='bestmodel') :: TrackerCallback`

TrackerCallback可在监控数量最佳时保存模型。

此回调在学习培训期间跟踪监视器中的数量。 模式可以强制为“最小”或“最大”，但会自动尝试确定数量应尽可能低（验证损失）还是最高可能（精确度）。 每当由每个（'改进'或'纪元'）确定时，将以名义保存模型。 在训练结束时加载最佳模型是每次''改进'。

In [6]:
model = simple_cnn((3,16,16,2))
learn = Learner(data, model, metrics=[accuracy])
learn.fit_one_cycle(5,1e-4, callbacks=[SaveModelCallback(learn, every='epoch', monitor='accuracy', name='model')])

epoch,train_loss,valid_loss,accuracy,time
0,0.682584,0.674573,0.753680,01:03
1,0.570130,0.518658,0.917076,01:03
2,0.371998,0.331077,0.936212,01:01
3,0.278062,0.266117,0.937684,01:00
4,0.258674,0.257299,0.937193,01:05


选择every='epoch'可以在每个时代结束时保存一个单独的模型。

In [12]:
(path/'models').ls()

[WindowsPath('C:/Users/Yang/.fastai/data/mnist_sample/models/bestmodel_0.pth'),
 WindowsPath('C:/Users/Yang/.fastai/data/mnist_sample/models/bestmodel_1.pth'),
 WindowsPath('C:/Users/Yang/.fastai/data/mnist_sample/models/bestmodel_2.pth'),
 WindowsPath('C:/Users/Yang/.fastai/data/mnist_sample/models/model_0.pth'),
 WindowsPath('C:/Users/Yang/.fastai/data/mnist_sample/models/model_1.pth'),
 WindowsPath('C:/Users/Yang/.fastai/data/mnist_sample/models/model_2.pth'),
 WindowsPath('C:/Users/Yang/.fastai/data/mnist_sample/models/model_3.pth'),
 WindowsPath('C:/Users/Yang/.fastai/data/mnist_sample/models/model_4.pth'),
 WindowsPath('C:/Users/Yang/.fastai/data/mnist_sample/models/tmp.pth'),
 WindowsPath('C:/Users/Yang/.fastai/data/mnist_sample/models/trained_model.pth')]

In [13]:
learn.fit_one_cycle(5,1e-4, callbacks=[SaveModelCallback(learn, every='improvement', monitor='accuracy', name='best')])

epoch,train_loss,valid_loss,accuracy,time
0,0.242412,0.221357,0.937193,01:04
1,0.183681,0.175380,0.937684,01:00
2,0.166774,0.163337,0.938665,01:00
3,0.160396,0.160284,0.939647,00:59
4,0.151034,0.159589,0.940137,00:59


Better model found at epoch 0 with accuracy value: 0.9371933341026306.
Better model found at epoch 1 with accuracy value: 0.9376839995384216.
Better model found at epoch 2 with accuracy value: 0.9386653304100037.
Better model found at epoch 3 with accuracy value: 0.9396467208862305.
Better model found at epoch 4 with accuracy value: 0.9401373863220215.


选择every='improvement'可以在训练期间保存所有时期中的单个最佳模型。

In [14]:
(path/'models').ls()

[WindowsPath('C:/Users/Yang/.fastai/data/mnist_sample/models/best.pth'),
 WindowsPath('C:/Users/Yang/.fastai/data/mnist_sample/models/bestmodel_0.pth'),
 WindowsPath('C:/Users/Yang/.fastai/data/mnist_sample/models/bestmodel_1.pth'),
 WindowsPath('C:/Users/Yang/.fastai/data/mnist_sample/models/bestmodel_2.pth'),
 WindowsPath('C:/Users/Yang/.fastai/data/mnist_sample/models/model_0.pth'),
 WindowsPath('C:/Users/Yang/.fastai/data/mnist_sample/models/model_1.pth'),
 WindowsPath('C:/Users/Yang/.fastai/data/mnist_sample/models/model_2.pth'),
 WindowsPath('C:/Users/Yang/.fastai/data/mnist_sample/models/model_3.pth'),
 WindowsPath('C:/Users/Yang/.fastai/data/mnist_sample/models/model_4.pth'),
 WindowsPath('C:/Users/Yang/.fastai/data/mnist_sample/models/tmp.pth'),
 WindowsPath('C:/Users/Yang/.fastai/data/mnist_sample/models/trained_model.pth')]

### Callback methods
你不要自己调用 - 他们会被fastai的Callback系统自动调用，以启用该类的功能。

<b>on_epoch_end</b>

`on_epoch_end(epoch:int, **kwargs:Any)`

将监控值与其最佳分数进行比较，并保存模型。

<b>on_train_end</b>

`on_train_end(**kwargs)`

加载最好的模型

### class ReduceLROnPlateauCallback

`ReduceLROnPlateauCallback(learn:Learner, monitor:str='valid_loss', mode:str='auto', patience:int=0, factor:float=0.2, min_delta:int=0) :: TrackerCallback`

TrackerCallback可在指标停止改进时降低学习率。

此回调在学习培训期间跟踪监视器中的数量。 模式可以强制为“最小”或“最大”，但会自动尝试确定数量应尽可能低（验证损失）还是最高可能（精确度）。 如果数量没有被min_delta提高，则会在耐心时期之后通过因子降低学习率。

### Callback methods
你不要自己调用 - 他们会被fastai的Callback系统自动调用，以启用该类的功能。

<b>on_train_begin</b>

`on_train_begin(**kwargs:Any)`

初始化内部参数。

<b>on_epoch_end</b>

`on_epoch_end(epoch, **kwargs:Any)`

将监控的值与其最佳值进行比较，并可能降低lr。

### class TrackerCallback

`TrackerCallback(learn:Learner, monitor:str='valid_loss', mode:str='auto') :: LearnerCallback`

LearnerCallback，用于跟踪监视器中的最佳值。

<b>get_monitor_value</b>

`get_monitor_value()`

选择监控值。

### Callback methods
你不要自己调用 - 他们会被fastai的Callback系统自动调用，以启用该类的功能。


<b>on_train_begin</b>

`on_train_begin(**kwargs:Any)`

初始化最佳值。